In [15]:
import numpy as np
import pandas as pd
import time
import gensim
import re
from gensim.models.word2vec import Word2Vec 
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [16]:
df = pd.read_csv('../adam/datasets/scrape_5.12.csv')

pd.set_option('display.max_colwidth', -1)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
df.head()

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start
0,1234258409408602118,e.p.c.,"Did the first of several shopping runs, though I ended up walking out of CVS without buying anything. If the rate of people coughing at me is any indication, we’ll all have Covid-19 by Thursday. \n\nCover your fucking mouths when you cough.",2020-03-01 23:25:10,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
1,1234253374725459968,@geminiwoe,Looks like adios muchachos for the US #COVID #COVID19US #coronavirushttps://twitter.com/nytimes/status/1234171621696557057 …,2020-03-01 23:05:10,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
2,1234241890700218370,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,Humour..snl...on #COVID #coronavirususahttps://mashable.com/video/snl-coronavirus-democratic-candidates-cold-open/ …,2020-03-01 22:19:32,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
3,1234238588474331136,° ° °,had to go to three diff convenient stores to find real milk (not soy) and they were all out of the lower priced non organic stuff .. i buy organic so i was fine with it but wow. covid 19 panic buying begins in manhattan.,2020-03-01 22:06:25,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
4,1234238537068883968,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,"#COVID #Coronovirius in nursing home setting , #Kirkland more than 50 with symptoms ..3/1/20https://www.statnews.com/2020/02/29/new-covid-19-death-raises-concerns-about-virus-spread-in-nursing-homes/ …",2020-03-01 22:06:12,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01


In [18]:
df.drop_duplicates(inplace=True)
df.shape

(114044, 10)

In [19]:
df['text'] = df['text'].astype(str).str.lower()
df['token_text'] = df['text'].str.replace('([^ a-zA-Z0-9])', '').str.replace('http\S+|www.\S+', '', case=False).replace('coronavirus', 'covid19')

stop = stopwords.words('english')
df['stop_text'] = df['token_text'].apply(lambda x: [item for item in str(x).split() 
                                                    if item not in stop])



#df['split_text'] = df['token_text'].astype(str).str.lower().str.split()

In [20]:
df['text'].head()

0    did the first of several shopping runs, though i ended up walking out of cvs without buying anything. if the rate of people coughing at me is any indication, we’ll all have covid-19 by thursday. \n\ncover your fucking mouths when you cough.
1    looks like adios muchachos for the us #covid #covid19us #coronavirushttps://twitter.com/nytimes/status/1234171621696557057 …                                                                                                                    
2    humour..snl...on #covid #coronavirususahttps://mashable.com/video/snl-coronavirus-democratic-candidates-cold-open/ …                                                                                                                            
3    had to go to three diff convenient stores to find real milk (not soy) and they were all out of the lower priced non organic stuff .. i buy organic so i was fine with it but wow. covid 19 panic buying begins in manhattan.                    
4    #covid #cor

In [21]:
df['stop_text'].head()

0    [first, several, shopping, runs, though, ended, walking, cvs, without, buying, anything, rate, people, coughing, indication, well, covid19, thursday, cover, fucking, mouths, cough]
1    [looks, like, adios, muchachos, us, covid, covid19us, coronavirus]                                                                                                                  
2    [humoursnlon, covid, coronavirususa]                                                                                                                                                
3    [go, three, diff, convenient, stores, find, real, milk, soy, lower, priced, non, organic, stuff, buy, organic, fine, wow, covid, 19, panic, buying, begins, manhattan]              
4    [covid, coronovirius, nursing, home, setting, kirkland, 50, symptoms, 3120]                                                                                                         
Name: stop_text, dtype: object

In [22]:
df['token_text'].head()

0    did the first of several shopping runs though i ended up walking out of cvs without buying anything if the rate of people coughing at me is any indication well all have covid19 by thursday cover your fucking mouths when you cough
1    looks like adios muchachos for the us covid covid19us coronavirus                                                                                                                                                                    
2    humoursnlon covid coronavirususa                                                                                                                                                                                                     
3    had to go to three diff convenient stores to find real milk not soy and they were all out of the lower priced non organic stuff  i buy organic so i was fine with it but wow covid 19 panic buying begins in manhattan               
4    covid coronovirius in nursing home setting  kirkland mo

In [23]:
sent = [row for row in df['stop_text']]

In [24]:
tweet_w2v = Word2Vec(size=1000, min_count=500, window=10)

In [25]:
tweet_w2v.build_vocab(sent)

In [28]:
tweet_w2v.most_similar('covid')

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('using', 0.10611840337514877),
 ('purity', 0.09726481139659882),
 ('ap', 0.08880230784416199),
 ('jersey', 0.08675691485404968),
 ('virus', 0.08607183396816254),
 ('states', 0.08507302403450012),
 ('f', 0.08471773564815521),
 ('rise', 0.08338930457830429),
 ('wouldnt', 0.08218280225992203),
 ('walking', 0.08204132318496704)]